<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hand-made-agent" data-toc-modified-id="Hand-made-agent-1">Hand made agent</a></span><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1.1">Goal</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-1.2">Imports</a></span></li><li><span><a href="#Simple-agent" data-toc-modified-id="Simple-agent-1.3">Simple agent</a></span><ul class="toc-item"><li><span><a href="#Code" data-toc-modified-id="Code-1.3.1">Code</a></span></li><li><span><a href="#Match-visualization" data-toc-modified-id="Match-visualization-1.3.2">Match visualization</a></span></li></ul></li><li><span><a href="#Research-agent" data-toc-modified-id="Research-agent-1.4">Research agent</a></span><ul class="toc-item"><li><span><a href="#Code" data-toc-modified-id="Code-1.4.1">Code</a></span></li><li><span><a href="#Match-visualization" data-toc-modified-id="Match-visualization-1.4.2">Match visualization</a></span></li></ul></li><li><span><a href="#Build-new-city-tiles" data-toc-modified-id="Build-new-city-tiles-1.5">Build new city tiles</a></span></li><li><span><a href="#Build-new-workers" data-toc-modified-id="Build-new-workers-1.6">Build new workers</a></span></li><li><span><a href="#How-I-would-play-as-a-human?" data-toc-modified-id="How-I-would-play-as-a-human?-1.7">How I would play as a human?</a></span></li><li><span><a href="#Challenges-found-when-creating-an-agent" data-toc-modified-id="Challenges-found-when-creating-an-agent-1.8">Challenges found when creating an agent</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.9">Summary</a></span></li><li><span><a href="#TODO" data-toc-modified-id="TODO-1.10">TODO</a></span></li></ul></li></ul></div>

# Hand made agent

## Goal

## Imports

In [1]:
import os
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from kaggle_environments import make
from functools import partial

from luxai.utils import render_game_in_html

Loading environment football failed: No module named 'gfootball'


In [2]:
from kaggle_environments.envs.lux_ai_2021.test_agents.python.lux.game import Game
from kaggle_environments.envs.lux_ai_2021.test_agents.python.lux.game_map import Cell, RESOURCE_TYPES
from kaggle_environments.envs.lux_ai_2021.test_agents.python.lux.constants import Constants
from kaggle_environments.envs.lux_ai_2021.test_agents.python.lux.game_constants import GAME_CONSTANTS
from kaggle_environments.envs.lux_ai_2021.test_agents.python.lux import annotate

In [3]:
plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (20, 5)  
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## Simple agent

I will first refactorize the simple agent to ease up the improvement process.

### Code

In [4]:
class BaseAgent():
    def __init__(self):
        self.game_state = None
        
    def _update_game_state(self, observation):
        if observation["step"] == 0:
            self.game_state = Game()
            self.game_state._initialize(observation["updates"])
            self.game_state._update(observation["updates"][2:])
            self.game_state.id = observation.player
        else:
            self.game_state._update(observation["updates"])
        
    def __call__(self, observation, configuration):
        self._update_game_state(observation)
        raise NotImplementedError('You have to implement this function')

In [5]:
import math

class SimpleAgent(BaseAgent):
    def __init__(self):
        super().__init__()


            
    def __call__(self, observation, configuration):
        self._update_game_state(observation)
        
        ### AI Code goes down here! ### 
        player = self.game_state.players[observation.player]
        #opponent = game_state.players[(observation.player + 1) % 2]
        resource_tiles = get_resource_tiles(self.game_state)
        actions = []
        # we iterate over all our units and do something with them
        for unit in get_available_workers(player):
            if unit.get_cargo_space_left() > 0:
                actions.append(move_to_closest_resource(unit, player, resource_tiles))
            else:
                actions.append(move_to_closest_city(unit, player))
                
        actions = [action for action in actions if action is not None]
        return actions
    
def get_resource_tiles(game_state):
    resource_tiles: list[Cell] = []
    for y in range(game_state.map.height):
        for x in range(game_state.map.width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles
    
def get_available_workers(player):
    return [unit for unit in player.units if unit.is_worker() and unit.can_act()]
    
def move_to_closest_resource(unit, player, resource_tiles):
    # if the unit is a worker and we have space in cargo, lets find the nearest resource tile and try to mine it
    closest_dist = math.inf
    closest_resource_tile = None
    for resource_tile in resource_tiles:
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
        dist = resource_tile.pos.distance_to(unit.pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    if closest_resource_tile is not None:
        return unit.move(unit.pos.direction_to(closest_resource_tile.pos))

def move_to_closest_city(unit, player):
    if len(player.cities) > 0:
        closest_dist = math.inf
        closest_city_tile = None
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(unit.pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
        if closest_city_tile is not None:
            move_dir = unit.pos.direction_to(closest_city_tile.pos)
            return unit.move(move_dir)

### Match visualization

In [6]:
env = make("lux_ai_2021", debug=True, configuration={'width': 12, 'height': 12, 'seed': 0, 'episodeSteps': 361})
initial_info = env.reset()
game_info = env.run(['simple_agent', SimpleAgent()])
#render_game_in_html(env)

In [7]:
render_game_in_html(env)

Opening in existing browser session.


## Research agent

Let's start by improving the simple agent to do research whith the available city tiles. I will first refactorize the simple agent to ease up the improvement process.

### Code

### Match visualization

## Build new city tiles

## Build new workers

## How I would play as a human?

## Challenges found when creating an agent

Let's enumerate the difficulties we have found when trying to create an agent by hand.

## Summary

## TODO

- Move code to the library
- Add tests 
- Function that annotates the resources of the map